# Train a Model for Detecting Cars

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/train_car_detection.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [3]:
pip install geoai-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.9/567.9 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.7/286.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883

## Import libraries

In [4]:
import geoai

## Download sample data

In [5]:
train_raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/cars_7cm.tif"
)
train_vector_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/car_detection.geojson"
test_raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/cars_test_7cm.tif"
)

In [6]:
train_raster_path = geoai.download_file(train_raster_url)
train_vector_path = geoai.download_file(train_vector_url)
test_raster_path = geoai.download_file(test_raster_url)

cars_7cm.tif: 100%|██████████| 92.0M/92.0M [00:01<00:00, 52.1MB/s]
car_detection.geojson: 100%|██████████| 359k/359k [00:00<00:00, 26.0MB/s]
cars_test_7cm.tif: 100%|██████████| 45.8M/45.8M [00:00<00:00, 72.5MB/s]


## Visualize sample data

In [7]:
geoai.view_vector_interactive(train_vector_path, tiles=train_raster_url)

In [8]:
geoai.view_raster(test_raster_url)

## Create training data

In [9]:
out_folder = "output"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_vector_path,
    tile_size=512,
    stride=256,
    buffer_radius=0,
)


Raster info for cars_7cm.tif:
  CRS: EPSG:3857
  Dimensions: 8351 x 4463
  Resolution: (0.07464844311342499, 0.07464346097829833)
  Bands: 3
  Bounds: BoundingBox(left=-10622651.27699905, bottom=3462200.907507864, right=-10622027.88785061, top=3462534.04127421)
Loaded 963 features from car_detection.geojson
Vector CRS: EPSG:4326
Reprojecting features from EPSG:4326 to EPSG:3857


Generated: 544, With features: 462: 100%|██████████| 544/544 [00:35<00:00, 15.21it/s]


------- Export Summary -------
Total tiles exported: 544
Tiles with features: 462 (84.9%)
Average feature pixels per tile: 23885.1
Output saved to: output

------- Georeference Verification -------


## Train object detection model

In [ ]:
geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models",
    num_channels=3,
    pretrained=True,
    batch_size=4,
    num_epochs=100,
    learning_rate=0.005,
    val_split=0.2,
)

Using device: cuda
Found 544 image files and 544 label files
Training on 435 images, validating on 109 images


Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:00<00:00, 182MB/s]


Epoch: 0, Batch: 0/109, Loss: 2.2843, Time: 3.94s
Epoch: 0, Batch: 10/109, Loss: 1.3727, Time: 9.00s
Epoch: 0, Batch: 20/109, Loss: 0.7256, Time: 10.37s
Epoch: 0, Batch: 30/109, Loss: 0.4613, Time: 9.83s
Epoch: 0, Batch: 40/109, Loss: 0.4260, Time: 9.78s
Epoch: 0, Batch: 50/109, Loss: 0.4752, Time: 9.81s
Epoch: 0, Batch: 60/109, Loss: 0.2151, Time: 9.44s
Epoch: 0, Batch: 70/109, Loss: 0.4739, Time: 9.97s
Epoch: 0, Batch: 80/109, Loss: 0.4642, Time: 9.98s
Epoch: 0, Batch: 90/109, Loss: 0.4082, Time: 10.44s
Epoch: 0, Batch: 100/109, Loss: 0.3623, Time: 10.11s
Epoch 1/100: Train Loss: 0.6211, Val Loss: inf, Val IoU: 0.6808
Saving best model with IoU: 0.6808
Epoch: 1, Batch: 0/109, Loss: 0.5580, Time: 2.60s
Epoch: 1, Batch: 10/109, Loss: 0.2789, Time: 10.52s
Epoch: 1, Batch: 20/109, Loss: 0.5184, Time: 10.00s
Epoch: 1, Batch: 30/109, Loss: 0.3641, Time: 9.93s
Epoch: 1, Batch: 40/109, Loss: 0.3106, Time: 10.23s
Epoch: 1, Batch: 50/109, Loss: 0.4566, Time: 10.77s
Epoch: 1, Batch: 60/109, Los

## Run inference

In [ ]:
masks_path = "cars_prediction.tif"
model_path = f"{out_folder}/models/best_model.pth"

In [ ]:
geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=512,
    overlap=256,
    confidence_threshold=0.5,
    batch_size=4,
    num_channels=3,
)

## Vectorize masks

In [ ]:
output_path = "cars_prediction.geojson"
gdf = geoai.orthogonalize(masks_path, output_path, epsilon=2)

## Visualize results

In [ ]:
geoai.view_vector_interactive(output_path, tiles=test_raster_url)

In [ ]:
geoai.create_split_map(
    left_layer=output_path,
    right_layer=test_raster_url,
    left_args={"style": {"color": "red", "fillOpacity": 0.2}},
    basemap=test_raster_url,
)

![image](https://github.com/user-attachments/assets/31f4eb34-b829-423d-a9a6-86fd79219770)